<a href="https://colab.research.google.com/github/KoroshRH/InceptionV3-transfer-learning/blob/main/transfer_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup the InceptionV3 model

To setup this model, we need to define the model and set these sections in it's constructor method:

1. Set the input shape to fit your application.

2. Pick and freeze the convolution layers to take advantage of the features it has learned already.

3. Add dense layers which you will train.

In the end, we have to download it's pretrained weights and load them.

In [ ]:
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers

pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = None)

In [ ]:
local_weights_file = '/tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
  layer.trainable = False

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5 \
    -O /tmp/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5

In this section, we choose `mixed7` layer as our last layer because it's not so specialized and we can customize it based on our application.

In [ ]:
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

## Add dense layers for your classifier

Next, we will add our layers to your model. These will be the layers that we want to train. We add dropout layer for preventing the overfitting.

In [ ]:
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Model

x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense  (1, activation='sigmoid')(x)           

model = Model(pre_trained_model.input, x) 

model.compile(optimizer = RMSprop(learning_rate=0.0001), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

## Prepare the dataset

Here we want to test **Cat vs Dog** dataset, but every other dataset can be tested with this model. We just have to adjust our newly added layers to be compatible to our the dataset needs. We are using 150x150 images for this dataset and this model's input.

In [ ]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator

!wget https://storage.googleapis.com/tensorflow-1-public/course2/cats_and_dogs_filtered.zip

zip_ref = zipfile.ZipFile("./cats_and_dogs_filtered.zip", 'r')
zip_ref.extractall("tmp/")
zip_ref.close()

base_dir = 'tmp/cats_and_dogs_filtered'

train_dir = os.path.join( base_dir, 'train')
validation_dir = os.path.join( base_dir, 'validation')

train_cats_dir = os.path.join(train_dir, 'cats') 

train_dogs_dir = os.path.join(train_dir, 'dogs') 

validation_cats_dir = os.path.join(validation_dir, 'cats') 

validation_dogs_dir = os.path.join(validation_dir, 'dogs')

# In this part of the code, we settup our data augmentation parameters to get better data distribution
train_datagen = ImageDataGenerator(rescale = 1./255.,
                                   width_shift_range = 0.25,
                                   height_shift_range = 0.25,
                                   shear_range = 0.25,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator( rescale = 1.0/255. )

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size = 20,
                                                    class_mode = 'binary', 
                                                    target_size = (150, 150))     

validation_generator =  test_datagen.flow_from_directory( validation_dir,
                                                          batch_size  = 20,
                                                          class_mode  = 'binary', 
                                                          target_size = (150, 150))

## Train the model

Finally, we train our model for 15 epochs and then we evaluate the results to see what happend through the training procedure.

In [ ]:
history = model.fit(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 15, validation_steps = 50, verbose = 2)

## Evaluate the results

You will use the same code to plot the results. As you can see, the validation accuracy is also trending upwards as your training accuracy improves. This is a good sign that your model is no longer overfitting!

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Testing accuracy')
plt.title('Training and Testing accuracy')
plt.legend(loc=0)
plt.figure()

plt.show()